Setup library

In [9]:
import duckdb

duckdb Setup

In [10]:
con = duckdb.connect()
con.sql("INSTALL httpfs; LOAD httpfs;")

Setup File Path & Object Name di MinIO

In [11]:
localPath = {
    'jakarta': r'C:/Users/Asus/Downloads/Data Andalan Utama (Intern)/Exercise_Data_Sekolah/cleanedData/dapodik/dataYangSudahDiKategorisasi.csv',
    'bekasi': r'C:/Users/Asus/Downloads/Data Andalan Utama (Intern)/Exercise_Data_Sekolah_2/matchData/revisi/part2/hasilAkhir/Bekasi.csv',
    'depok': r'C:/Users/Asus/Downloads/Data Andalan Utama (Intern)/Exercise_Data_Sekolah_2/matchData/revisi/part2/hasilAkhir/Depok.csv',
    'balikpapan': r'C:/Users/Asus/Downloads/Data Andalan Utama (Intern)/Exercise_Data_Sekolah_3/afterCategorized/bekasiMatchCategorized.csv',
    'makassar': r'C:/Users/Asus/Downloads/Data Andalan Utama (Intern)/Exercise_Data_Sekolah_3/afterCategorized/makassarMatchCategorized.csv',
    'palembang': r'C:/Users/Asus/Downloads/Data Andalan Utama (Intern)/Exercise_Data_Sekolah_3/afterCategorized/palembangMatchCategorized.csv'
}

minioObjectName = {
    'jakarta': 'staging/parquet/duckdb/JawaBarat/jakarta.parquet',
    'bekasi': 'staging/parquet/duckdb/JawaBarat/bekasi.parquet',
    'depok': 'staging/parquet/duckdb/JawaBarat/depok.parquet',
    'balikpapan': 'staging/parquet/duckdb/kalimantanTimur/balikpapan.parquet',
    'makassar': 'staging/parquet/duckdb/sulawesiSelatan/makassar.parquet',
    'palembang': 'staging/parquet/duckdb/sumateraSelatan/palembang.parquet'
}

targetFilePath = {
    'jakarta': 'parquetDuckDBLocalTemp/jakarta.parquet',
    'bekasi': 'parquetDuckDBLocalTemp/bekasi.parquet',
    'depok': 'parquetDuckDBLocalTemp/depok.parquet',
    'balikpapan': 'parquetDuckDBLocalTemp/balikpapan.parquet',
    'makassar': 'parquetDuckDBLocalTemp/makassar.parquet',
    'palembang': 'parquetDuckDBLocalTemp/palembang.parquet'
}

convert csv to parquet

In [12]:
jakartaParquet = con.execute(f"""
            COPY             
            (SELECT * FROM read_csv_auto('{localPath['jakarta']}', AUTO_DETECT=TRUE))

            TO
            '{targetFilePath['jakarta']}' (FORMAT 'PARQUET');
            """)
depokParquet = con.execute(f"""
            COPY             
            (SELECT * FROM read_csv_auto('{localPath['depok']}', AUTO_DETECT=TRUE))

            TO
            '{targetFilePath['depok']}' (FORMAT 'PARQUET');
            """)
bekasiParquet = con.execute(f"""
            COPY             
            (SELECT * FROM read_csv_auto('{localPath['bekasi']}', AUTO_DETECT=TRUE))

            TO
            '{targetFilePath['bekasi']}' (FORMAT 'PARQUET');
            """)
balikpapanParquet = con.execute(f"""
            COPY             
            (SELECT * FROM read_csv_auto('{localPath['balikpapan']}', AUTO_DETECT=TRUE))

            TO
            '{targetFilePath['balikpapan']}' (FORMAT 'PARQUET');
            """)
makassarParquet = con.execute(f"""
            COPY             
            (SELECT * FROM read_csv_auto('{localPath['makassar']}', AUTO_DETECT=TRUE))

            TO
            '{targetFilePath['makassar']}' (FORMAT 'PARQUET');
            """)
palembangParquet = con.execute(f"""
            COPY             
            (SELECT * FROM read_csv_auto('{localPath['palembang']}', AUTO_DETECT=TRUE))

            TO
            '{targetFilePath['palembang']}' (FORMAT 'PARQUET');
            """)


MinIO Setup

In [13]:
minioAuth = {
    'accessKey': 'minioadmin',
    'secretKey': 'miniopassword',
    'bucket': 'data-dapodik-kemendikdasmen',
    'endpoint': 'http://localhost:9000'
}

Minio Connection

In [14]:
con.sql(f"SET s3_endpoint = '{minioAuth['endpoint'].replace('http://', '')}';")
con.sql("SET s3_use_ssl = false;") 
con.sql(f"SET s3_access_key_id = '{minioAuth['accessKey']}';")
con.sql(f"SET s3_secret_access_key = '{minioAuth['secretKey']}';")
con.sql("SET s3_url_style = 'path';") # WAJIB untuk MinIO/Local

Full EndPoint URL

In [15]:
fullEndPointURL = {
    'jakarta': f"s3://{minioAuth['bucket']}/{minioObjectName['jakarta']}",
    'bekasi': f"s3://{minioAuth['bucket']}/{minioObjectName['bekasi']}",
    'depok': f"s3://{minioAuth['bucket']}/{minioObjectName['depok']}",
    'balikpapan': f"s3://{minioAuth['bucket']}/{minioObjectName['balikpapan']}",
    'makassar': f"s3://{minioAuth['bucket']}/{minioObjectName['makassar']}",
    'palembang': f"s3://{minioAuth['bucket']}/{minioObjectName['palembang']}"
}

In [16]:
loadJakarta = f"""
    COPY 
        (
            SELECT *
            FROM 
                read_parquet('{targetFilePath['jakarta']}')
        ) 
    TO 
        '{fullEndPointURL['jakarta']}'
    (FORMAT 'PARQUET', 
     OVERWRITE TRUE);
"""
loadBekasi = f"""
    COPY 
        (
            SELECT *
            FROM 
                read_parquet('{targetFilePath['bekasi']}')
        ) 
    TO 
        '{fullEndPointURL['bekasi']}'
    (FORMAT 'PARQUET', 
     OVERWRITE TRUE);
"""
loadDepok = f"""
    COPY 
        (
            SELECT *
            FROM 
                read_parquet('{targetFilePath['depok']}')
        ) 
    TO 
        '{fullEndPointURL['depok']}'
    (FORMAT 'PARQUET', 
     OVERWRITE TRUE);
"""
loadBalikpapan = f"""
    COPY 
        (
            SELECT *
            FROM 
                read_parquet('{targetFilePath['balikpapan']}')
        ) 
    TO 
        '{fullEndPointURL['balikpapan']}'
    (FORMAT 'PARQUET', 
     OVERWRITE TRUE);
"""
loadMakassar = f"""
    COPY 
        (
            SELECT *
            FROM 
                read_parquet('{targetFilePath['makassar']}')
        ) 
    TO 
        '{fullEndPointURL['makassar']}'
    (FORMAT 'PARQUET', 
     OVERWRITE TRUE);
"""
loadPalembang = f"""
    COPY 
        (
            SELECT *
            FROM 
                read_parquet('{targetFilePath['palembang']}')
        ) 
    TO 
        '{fullEndPointURL['palembang']}'
    (FORMAT 'PARQUET', 
     OVERWRITE TRUE);
"""

In [17]:
# --- 2. Jalankan Proses Migrasi ---
try:
    con.sql(loadJakarta)
    con.sql(loadBekasi)
    con.sql(loadDepok)
    con.sql(loadBalikpapan)
    con.sql(loadMakassar)
    con.sql(loadPalembang)
    
    print(f"\n✅ Sukses! Data CSV telah dibaca, dikonversi ke Parquet, dan dikirim langsung ke MinIO")

except Exception as e:
    print(f"\n❌ Terjadi error saat migrasi: {e}")


✅ Sukses! Data CSV telah dibaca, dikonversi ke Parquet, dan dikirim langsung ke MinIO


In [18]:
con.close()